In [3]:
#PART 1: READING THE TEXT FILES FOR JOYCE'S WORKS


filenames = ['Ulysses.txt', 'Portrait.txt', 'Dubliners.txt']

# dictionary to hold the content of each file
texts = {}

# loop through each file
for filename in filenames:
    # Open each file
    with open(filename, 'r', encoding='utf-8') as file:
        # Read the content and store it in the dictionary
        texts[filename] = file.read()

FileNotFoundError: [Errno 2] No such file or directory: 'Ulysses.txt'

In [ ]:
#PART 2: INITIAL EXPLORATION OF DATA

word_counts = {filename: len(texts[filename].split()) for filename in texts}
print(word_counts)

from collections import Counter
most_common_words = {filename: Counter(texts[filename].split()).most_common(100) for filename in texts}
print(most_common_words) 

In [1]:
#PART 3: USING NLP LIBRARY SPACY TO EXTRACT ENTITIES // PART 4: GEOCODING THOSE ENTITIES AND SAVING TO .JSON

import spacy
import requests
import json

spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 2000000  


# Dictionary to store geocoded locations
geocoded_results = {}

# Words to exclude
excluded_words = {'Leopold', 'Bloom', 'Molly', 'Bloom', 'Curran', 'Pharaohs',
                   'Ursula', 'hockey', 'ivory', 'continent',
                  'Stephen', 'Dedalus', 'Buck', 'Mulligan', 'Haines', 'Gerty',
                    'MacDowell', 'Simon', 'Dedalus',
                  'Milly', 'Bloom', 'Blazes', 'Boylan', 'Marion',
                    'Bloom', 'Stephen', 'Dedalus', 'Simon', 'Dedalus',
                  'Mary', 'Dedalus', 'Dante', 'Riordan', 'Cranly', 'Davin', 
                  'Lynch', 'Emma', 'Clery', 'Gabriel',
                  'Conroy', 'Gretta', 'Conroy', 'Eveline', 'Hill', 'Little',
                    'Chandler', 'Duffy', 'Maria', 'Farrington',
                  'Sinico', 'Kernan', 'Lenehan', 'Dignam', 'Fitzgibbon',
                    'Heseltine', 'Solemn', 'quae', 'bangbang', 'malachi',
                    'Boody', 'Thornton', 'Kiar', 'Victoria', 'Malaria',
                      'Cummins', 'Gael', 'Sambo',
                   'Vincent',  'Heenan', 'Leahy', 'DR', 'Maggy', 'Alas',
                     'Marsh', 'Swill',
                     'reek', 'Andrews', 'Helen', 'Julia', 'Egan', 'Lower',
                       'Temple', 'Duffy'}


def geocode_location(location):
    url = f"https://nominatim.openstreetmap.org/search?q={location}&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data:
            return {
                "location": location,
                "latitude": float(data[0]['lat']),
                "longitude": float(data[0]['lon']),
                "address": data[0]['display_name']
            }
    return None


for filename in filenames:
    with open(filename, 'r', encoding='utf-8') as file:
        texts[filename] = file.read()

  
    doc = nlp(texts[filename])

    geocoded_results[filename] = []

   
    locations = [ent.text for ent in doc.ents if ent.label_ in ['GPE', 'LOC'] and ent.text not in excluded_words]
    for location in locations:
        geocoded_location = geocode_location(location)
        if geocoded_location:
            geocoded_results[filename].append(geocoded_location)


with open('geocoded_locations.json', 'w', encoding='utf-8') as json_file:
    json.dump(geocoded_results, json_file, ensure_ascii=False, indent=4)

print("Geocoding complete. Data saved to 'geocoded_locations.json'.")

NameError: name 'filenames' is not defined